# Машинное обучение ранжированию с помощью библиотеки XGBoost

В этом примере мы:

- познакомимся с библиотекой **XGBoost**
- научимся решать задачу машинного обучения ранжирования используя алгоритмы, реализованные в **XGBoost**

Будем использовать стандартный датасет [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)

Подробности про формат датасета можно найти в соседнем тюториале *catboost_ltr.ipynb* по машинному обучению ранжированию с помощью библиотеки **CatBoost**, предполагаем что мы его уже прошли.

## Библиотека XGBoost

Полезные ссылки:
- Вся документация: https://xgboost.readthedocs.io/en/stable/
- Тюториал по learning to rank: https://xgboost.readthedocs.io/en/latest/tutorials/learning_to_rank.html
- Полноценный пример: https://xgboost.readthedocs.io/en/latest/python/examples/learning_to_rank.html

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [2]:
import copy
from timeit import default_timer as timer

import xgboost as xgb
# We'll use catboost to download dataset and to compute metrics
from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим встроенный в катбуст сабсет датасета MSLR:

In [3]:
df_train, df_test = datasets.msrank_10k()

Для удобства присвоим колонкам говорящие имена:

In [4]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns

# Assign column names
columns = generate_column_names(num_features=136)
df_train.columns = columns
df_test.columns = columns

Теперь наши данные выглядят красивее:

In [5]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0    1          3          3          0          0          3   
1    2.0    1          3          0          3          0          3   
2    0.0    1          3          0          2          0          3   
3    2.0    1          3          0          3          0          3   
4    1.0    1          3          0          3          0          3   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...           62     11089534   
1        1.0        0.0   1.000000  ...           54     11089534   
2        1.0        0.0   0.666667  ...           45            3   
3        1.0        0.0   1.000000  ...           56     11089534   
4        1.0        0.0   1.000000  ...           64            5   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0            2          116        64034           13            3   
1           

Всего у нас:

- 87 запросов и 10000 документов в трейне
- 88 запросов и 10000 документов в тесте
- 130 колонок: таргет (оценка асессора), ID запроса (qid) и вектор из 128 фичей
- таргет принимает значения в интервале \[0,4\]

Теперь нам надо представить датасет в формате, который можно подавать на вход модели xgboost.

По полной аналогии с тем как мы это делали для catboost, придется разделить его на 3 части:

- **y** -- вектор таргетов
- **X** -- тензор из фичей
- **q** -- вектор из ID запросов, которые позволяют сгруппировать все документы, которые относятся к одному и тому же запросу

Заметим, что xgboost, в отличие от catboost, требует строгой отсортированности по ID запроса. Однако в данном случае нам повезло и датасет уже и так отсортирован по qid.

Убедимся в этом:

In [6]:
print(df_train['qid'].is_monotonic_increasing)
print(df_test['qid'].is_monotonic_increasing)

True
True


Сконвертируем датасет в нужный формат:

In [7]:
def to_xgboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

X_train, y_train, q_train = to_xgboost_dataset(df_train)
X_test, y_test, q_test = to_xgboost_dataset(df_test)

## Обучаем pointwise модель

Теперь можно приступить непосредственно к обучению модели. Мы начнем с простой pointwise модели которая в качестве лосса использует обычное RMSE.

Количество деревьев будем выбирать автоматически с использованием т.н. early stopping -- процесс обучения будем остановлен после того, как на валидационном множестве перестанет расти наша целевая метрика.  
В качестве целевой метрики, в отличие от примера с catboost, будем использовать не NDCG@10 а RMSE т.к. *XGBRegressor* не имеет информации о группах.

Создадим объект модели:

In [8]:
model = xgb.XGBRegressor(
    n_estimators=1000,             # maximum possible number of trees
    objective='reg:squarederror',  # RMSE
    eval_metric=['rmse'],          # metric used for early stopping
    early_stopping_rounds=10,      # stop if eval_metric does not improve for N rounds
    random_state=22,
)

И зафитим ее на нашем обучающем множестве.  
Т.к. мы используем early stopping то передадим в функцию fit() в качестве валидационного множества (параметр eval_set) наш тест-сет (строго говоря так делать нельзя, но мы в данном примере не стремимся к строгости, подробнее этот момент расписан в тюториале про машинное обучение ранжированию с помощью catboost).

In [9]:
# Fit
start = timer()
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

[0]	validation_0-rmse:0.77005
[1]	validation_0-rmse:0.75590
[2]	validation_0-rmse:0.74978
[3]	validation_0-rmse:0.74625
[4]	validation_0-rmse:0.74403
[5]	validation_0-rmse:0.74302
[6]	validation_0-rmse:0.74237
[7]	validation_0-rmse:0.74230
[8]	validation_0-rmse:0.74380
[9]	validation_0-rmse:0.74397
[10]	validation_0-rmse:0.74430
[11]	validation_0-rmse:0.74684
[12]	validation_0-rmse:0.74729
[13]	validation_0-rmse:0.74711
[14]	validation_0-rmse:0.74812
[15]	validation_0-rmse:0.74948
[16]	validation_0-rmse:0.75021
[17]	validation_0-rmse:0.75092
Model fit: elapsed = 0.528


Посмотрим, сколько деревьев содержит модель. Это немного сложнее чем было с катбустом:

In [10]:
num_trees = len(model.get_booster().get_dump())
print(f"Model params: num_trees = {num_trees} best_iteration = {model.best_iteration} best_score = {model.best_score:.3f}")

Model params: num_trees = 18 best_iteration = 7 best_score = 0.742


Видно, что всего модель содержит 18 деревьев, но лучший скор на валидации достигается уже после применения 7 из них.  
Поэтому на этапе инференса будет использоваться всего 7 деревьев.

При желании, мы можем сохранить эту модель в формате json:

In [11]:
model_file = "/tmp/model.xgb.json"

# Save model
model.save_model(model_file)

# Load model
# model = xgb.XGBRegressor()
# model.load_model(model_file)

Получим предикты модели на тестовом множестве:

In [12]:
y_hat_test = model.predict(X_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Predicted: y_hat_test.shape = (10000,)


Теперь, имея предикты, можно посчитать метрики качества, тут для удобства и единообразия будем использовать catboost:

In [13]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp', 'DCG:top=10;type=Exp', 'MAP:top=10']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.383
metric = DCG:top=10;type=Exp score = 7.725
metric = MAP:top=10 score = 0.487


Видим, что значение NDCG на тесте получилось значительно хуже, чем у катбуста (но, при желании, можно сделать сильно лучше если потюнить параметры модели).

## Обучаем pairwise модели

Теперь проделаем все то же самое, но с использованием алгоритма RankNet.  

Для этого вместо *XGBRegressor* будем использовать класс *XGBRanker*, которому в качестве objective передадим 'rank:pairwise'.  

Также, теперь мы:

- должны передавать в метод fit() наш вектор **q**, чтобы модель могла сгруппировать документы по запросу
- в качестве целевой метрики для early stopping можно использовать NDCG@10 а не RMSE

In [14]:
# Create model
model = xgb.XGBRanker(
    n_estimators=1000,          # maximum possible number of trees
    objective='rank:pairwise',  # RankNet objective
    eval_metric=["ndcg@10"],    # metric used for early stopping
    early_stopping_rounds=10,   # stop if eval_metric does not improve for N rounds
    random_state=22,
)

# Fit
start = timer()
model.fit(X_train, y_train, qid=q_train, eval_set=[(X_test, y_test)], eval_qid=[q_test], verbose=True)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} num_trees = {model.best_iteration}")

# Predict
y_hat_test = model.predict(X_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

# Compute metrics on test
print("\nEvaluated:")
compute_metrics(y_test, y_hat_test, q_test)

[0]	validation_0-ndcg@10:0.32002
[1]	validation_0-ndcg@10:0.32549
[2]	validation_0-ndcg@10:0.36129
[3]	validation_0-ndcg@10:0.35295
[4]	validation_0-ndcg@10:0.35903
[5]	validation_0-ndcg@10:0.36624
[6]	validation_0-ndcg@10:0.35605
[7]	validation_0-ndcg@10:0.36034
[8]	validation_0-ndcg@10:0.36515
[9]	validation_0-ndcg@10:0.36406
[10]	validation_0-ndcg@10:0.36592
[11]	validation_0-ndcg@10:0.37081
[12]	validation_0-ndcg@10:0.37719
[13]	validation_0-ndcg@10:0.37609
[14]	validation_0-ndcg@10:0.37223
[15]	validation_0-ndcg@10:0.37988
[16]	validation_0-ndcg@10:0.38136
[17]	validation_0-ndcg@10:0.38383
[18]	validation_0-ndcg@10:0.38399
[19]	validation_0-ndcg@10:0.38742
[20]	validation_0-ndcg@10:0.38553
[21]	validation_0-ndcg@10:0.38148
[22]	validation_0-ndcg@10:0.38043
[23]	validation_0-ndcg@10:0.37618
[24]	validation_0-ndcg@10:0.37409
[25]	validation_0-ndcg@10:0.37729
[26]	validation_0-ndcg@10:0.36836
[27]	validation_0-ndcg@10:0.36914
[28]	validation_0-ndcg@10:0.37111
Model fit: elapsed = 0.7

Сравним результаты:

- RMSE модель выбила NDCG@10 = **0.383**
- а RankNet выбивает уже NDCG@10 = **0.387**

Мы опять видим (совсем небольшое) преимущество listwise-подхода над "наивным" pointwise.

И, наконец, попробуем полноценный LambdaRank, для этого:

- в качестве objective будем использовать значение 'rank:ndcg'
- будем использовать дополнительные параметры lambdarank_pair_method и lambdarank_num_pair_per_sample (про них можно почитать в [документации](https://xgboost.readthedocs.io/en/latest/tutorials/learning_to_rank.html))

In [15]:
# Create model
model = xgb.XGBRanker(
    n_estimators=1000,          # maximum possible number of trees
    objective='rank:ndcg',      # LambdaRank objective
    lambdarank_pair_method='topk',
    lambdarank_num_pair_per_sample=20,
    eval_metric=["ndcg@10"],    # metric used for early stopping
    early_stopping_rounds=10,   # stop if eval_metric does not improve for N rounds
    random_state=22,
)

# Fit
start = timer()
model.fit(X_train, y_train, qid=q_train, eval_set=[(X_test, y_test)], eval_qid=[q_test], verbose=True)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} num_trees = {model.best_iteration}")

# Predict
y_hat_test = model.predict(X_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

# Compute metrics on test
print("\nEvaluated:")
compute_metrics(y_test, y_hat_test, q_test)

[0]	validation_0-ndcg@10:0.27110
[1]	validation_0-ndcg@10:0.34484
[2]	validation_0-ndcg@10:0.35342
[3]	validation_0-ndcg@10:0.36961
[4]	validation_0-ndcg@10:0.37509
[5]	validation_0-ndcg@10:0.38522
[6]	validation_0-ndcg@10:0.39158
[7]	validation_0-ndcg@10:0.39954
[8]	validation_0-ndcg@10:0.39468
[9]	validation_0-ndcg@10:0.39764
[10]	validation_0-ndcg@10:0.39399
[11]	validation_0-ndcg@10:0.39063
[12]	validation_0-ndcg@10:0.39061
[13]	validation_0-ndcg@10:0.39648
[14]	validation_0-ndcg@10:0.40214
[15]	validation_0-ndcg@10:0.40002
[16]	validation_0-ndcg@10:0.39901
[17]	validation_0-ndcg@10:0.39175
[18]	validation_0-ndcg@10:0.40457
[19]	validation_0-ndcg@10:0.40528
[20]	validation_0-ndcg@10:0.40556
[21]	validation_0-ndcg@10:0.41136
[22]	validation_0-ndcg@10:0.41160
[23]	validation_0-ndcg@10:0.41619
[24]	validation_0-ndcg@10:0.41421
[25]	validation_0-ndcg@10:0.40828
[26]	validation_0-ndcg@10:0.40973
[27]	validation_0-ndcg@10:0.41216
[28]	validation_0-ndcg@10:0.40768
[29]	validation_0-ndcg@1

Еще раз сравним результаты:

- RMSE модель выбила NDCG@10 = **0.383**
- RankNet выбивает уже NDCG@10 = **0.387**
- LambdaRank гораздо лучше всех и выбивает **0.416**

Однако, обратим внимание, что, несмотря на то, что pairwise подходы опять победили, абсолютные значения NDCG@10 даже у LambdaRank хуже того, что выбивала простая регрессия в catboost (**0.419**), не говоря уже от YetiRank!

Видно, что "из коробки" catboost показывает результаты значительно лучше, чем xgboost.  
Справедливости ради заметим, что мы в наших примерах совсем не пытались тюнить гиперпараметры -- если это проделать, то из xgboost можно "выжать" скоры значительно выше.  
Тем не менее, по нашему опыту, на данный момент catboost действительно является лучшим инструментом для решения задачи ранжирования.